## Reading Libraries 

In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
import nltk
from nltk.tokenize import MWETokenizer
import pandas as pd

In [ ]:
nltk.download('punkt')

In [30]:
lab_data = pd.read_csv('labeled_data.csv',nrows=5000)

In [17]:
unlab_data = unlab_data1[1:1000]

In [31]:
lab_data['text'][1]

'Flirted with giving this two stars, but that\'s a pretty damning rating for what might have just been an off night...\r\n\r\nNew to the East side, and so we don\'t know many of these hidden gems, but me and the fiance met her friend for drinks here and ended up getting some things to nibble. \r\n\r\nFirst off, service was pretty slow, which was unusual because the restaurant is pretty small and galley style. You would think it would be easy for servers to routinely hit up tables as you pass by. \r\n\r\nThe fiance ordered the Quinoa Salad, and said it was pretty good, but dry. I wasn\'t too hungry and so I simply ordered the Bruchetta 3-way which came with burnt crostinis. And I ordered a side of fries, which were either hard or chewy.\r\n\r\nThe friend ordered the macaroni & cheese, and added chicken and bacon (her usual order) and liked it.  \r\n\r\nCan\'t remember the last time I thought to myself- "Huh... they failed at fries..." So, like I said- two stars. But, the decor was good,

## Case Normalisation, Tokenization and Stop words removal

In [38]:
stopwords = []
with open('./stopwords_en.txt') as f:
    stopwords = f.read().splitlines()

In [39]:
def token(raw_data):
        #print(raw_data)
    #for raw_data in data:
        raw_data1 = raw_data.lower()
        tokenised = nltk.tokenize.word_tokenize(raw_data1)
        stopwords_tokens = [w for w in tokenised if not w in stopwords] 
        
        #print(tokenised)
        return(stopwords_tokens)

In [40]:
import multiprocessing as mp
pool = mp.Pool(processes=4)
tokenized_reuters = pool.map(token, lab_data['text'])
print(tokenized_reuters)


[['rule', '-', 'waiting', 'table', '(', ')', ',', 'wait', 'inside', '.', 'posted', 'sign', 'upfront', 'concerns', 'seated', 'patrons', '.', 'awful', '?', 'included', 'apology', '``', 'cold', "''", '.', 'p.s', 'calling', 'ahead', 'reserve', 'table', 'waiting', 'list', 'short', '.', ',', 'show', 'reserve', '.', 'boourns', '.', 'place', 'wrong', 'eyes', '.', 'rattle', 'equally-clever/witty-name-for', 'hot', 'beverage', '.', 'mention', '-', 'obsessed', 'mad', 'gab', '.', 'xoxo', '.'], ['flirted', 'giving', 'stars', ',', "'s", 'pretty', 'damning', 'rating', 'night', '...', 'east', 'side', ',', "n't", 'hidden', 'gems', ',', 'fiance', 'met', 'friend', 'drinks', 'ended', 'things', 'nibble', '.', ',', 'service', 'pretty', 'slow', ',', 'unusual', 'restaurant', 'pretty', 'small', 'galley', 'style', '.', 'easy', 'servers', 'routinely', 'hit', 'tables', 'pass', '.', 'fiance', 'ordered', 'quinoa', 'salad', ',', 'pretty', 'good', ',', 'dry', '.', "n't", 'hungry', 'simply', 'ordered', 'bruchetta', '3-